In [12]:
import os 
import pandas as pd


In [13]:
data = pd.read_csv('exprs_train.csv')

In [14]:
data.head()

,affy_hg_u133_plus_2,hgnc_symbol,gene_biotype,GSM1130907_Joo091-HG-U133Plus2.cel.gz,GSM1130908_Joo092-HG-U133Plus2.cel.gz,GSM1130909_Joo093-HG-U133Plus2.cel.gz,GSM1130910_Joo094-HG-U133Plus2.cel.gz,GSM1130911_Joo095-HG-U133Plus2.cel.gz,GSM1130912_Joo096-HG-U133Plus2.cel.gz,GSM1130913_Joo097-HG-U133Plus2.cel.gz,...,GSM1130953_Joo148-HG-U133Plus2.cel.gz,GSM1130954_Joo149-HG-U133Plus2.cel.gz,GSM1130955_Joo150-HG-U133Plus2.cel.gz,GSM1130956_Joo151-HG-U133Plus2.cel.gz,GSM1130957_Joo152-HG-U133Plus2.cel.gz,GSM1130958_Joo153-HG-U133Plus2.cel.gz,GSM1130959_Joo154-HG-U133Plus2.cel.gz,GSM1130960_Joo155-HG-U133Plus2.cel.gz,GSM1130961_Joo156-HG-U133Plus2.cel.gz,label
0,215559_at,ABCC6,protein_coding,3.387972,3.197034,3.066003,3.270180,3.483682,3.487162,2.901785,...,3.575743,3.382958,3.418691,3.125065,3.249330,3.413010,3.325482,3.458992,3.289809,1
1,214033_at,ABCC6,protein_coding,4.991891,5.131626,4.714632,4.880840,4.844942,4.833550,4.635570,...,5.068621,4.789166,4.754656,4.859572,4.854206,4.946681,4.862422,5.012400,4.610862,1
2,208480_s_at,ABCC6,protein_coding,4.204203,4.254020,4.781491,4.468411,4.432407,4.249169,4.588663,...,4.237186,4.442878,4.556832,4.827632,4.286801,4.573060,4.598085,4.873792,4.735709,1
3,207427_at,ACR,protein_coding,5.310477,5.301529,5.402414,5.248901,5.335763,5.170322,5.697215,...,5.471443,5.730578,5.318175,5.083529,5.393456,5.319998,5.289243,5.454217,5.741971,0
4,AFFX-HSAC07/X00351_M_at,ACTB,protein_coding,12.456092,11.877125,11.971389,12.726054,12.703934,12.901557,11.662964,...,12.796530,12.477227,12.732850,11.979345,12.830888,12.722848,12.252823,11.755974,12.576457,1


In [15]:
data.columns

Index(['affy_hg_u133_plus_2', 'hgnc_symbol', 'gene_biotype',
       'GSM1130907_Joo091-HG-U133Plus2.cel.gz',
       'GSM1130908_Joo092-HG-U133Plus2.cel.gz',
       'GSM1130909_Joo093-HG-U133Plus2.cel.gz',
       'GSM1130910_Joo094-HG-U133Plus2.cel.gz',
       'GSM1130911_Joo095-HG-U133Plus2.cel.gz',
       'GSM1130912_Joo096-HG-U133Plus2.cel.gz',
       'GSM1130913_Joo097-HG-U133Plus2.cel.gz',
       'GSM1130914_Joo098-HG-U133Plus2.cel.gz',
       'GSM1130915_Joo099-HG-U133Plus2.cel.gz',
       'GSM1130916_Joo100-HG-U133Plus2.cel.gz',
       'GSM1130917_Joo101-HG-U133Plus2.cel.gz',
       'GSM1130918_Joo102-HG-U133Plus2.cel.gz',
       'GSM1130919_Joo109-HG-U133Plus2.cel.gz',
       'GSM1130920_Joo110-HG-U133Plus2.cel.gz',
       'GSM1130921_Joo111-HG-U133Plus2.cel.gz',
       'GSM1130922_Joo113-HG-U133Plus2.cel.gz',
       'GSM1130923_Joo114-HG-U133Plus2.cel.gz',
       'GSM1130924_Joo115-HG-U133Plus2.cel.gz',
       'GSM1130925_Joo116-HG-U133Plus2.cel.gz',
       'GSM1130926_Joo118-H

In [16]:
data.shape

(944, 59)

In [17]:
data.nunique()

affy_hg_u133_plus_2                      880
hgnc_symbol                              376
gene_biotype                               3
GSM1130907_Joo091-HG-U133Plus2.cel.gz    864
GSM1130908_Joo092-HG-U133Plus2.cel.gz    872
GSM1130909_Joo093-HG-U133Plus2.cel.gz    872
GSM1130910_Joo094-HG-U133Plus2.cel.gz    872
GSM1130911_Joo095-HG-U133Plus2.cel.gz    875
GSM1130912_Joo096-HG-U133Plus2.cel.gz    864
GSM1130913_Joo097-HG-U133Plus2.cel.gz    867
GSM1130914_Joo098-HG-U133Plus2.cel.gz    873
GSM1130915_Joo099-HG-U133Plus2.cel.gz    871
GSM1130916_Joo100-HG-U133Plus2.cel.gz    870
GSM1130917_Joo101-HG-U133Plus2.cel.gz    868
GSM1130918_Joo102-HG-U133Plus2.cel.gz    865
GSM1130919_Joo109-HG-U133Plus2.cel.gz    869
GSM1130920_Joo110-HG-U133Plus2.cel.gz    870
GSM1130921_Joo111-HG-U133Plus2.cel.gz    860
GSM1130922_Joo113-HG-U133Plus2.cel.gz    863
GSM1130923_Joo114-HG-U133Plus2.cel.gz    862
GSM1130924_Joo115-HG-U133Plus2.cel.gz    870
GSM1130925_Joo116-HG-U133Plus2.cel.gz    871
GSM1130926